In [1]:
import tabula

file_name = 'NSTU_AMATH.pdf'

# Read pdf into list of DataFrame
dfs = tabula.read_pdf(file_name, pages='all')

# Read remote pdf into list of DataFrame
dfs2 = tabula.read_pdf("https://github.com/tabulapdf/tabula-java/raw/master/src/test/resources/technology/tabula/arabic.pdf")

# convert PDF into CSV file
tabula.convert_into(file_name, "output.csv", output_format="csv", pages='all')

# convert all PDFs in a directory
# tabula.convert_into_by_batch("input_directory", output_format='csv', pages='all')

Got stderr: Mar 09, 2023 10:00:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 09, 2023 10:00:06 PM org.apache.pdfbox.pdmodel.f

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('output.csv', )
df.head()


,Course Code,Unnamed: 1,Course Title,Credits,Contact hours\rper week,Duration of\rExamination(hrs)
0,A.MTH- 1101,NaN,Basic Algebra,3,3,4
1,A.MTH -1103,NaN,Set Theory,3,3,4
2,A.MTH -1105,NaN,Calculus-I,3,3,4
3,A.MTH- 1107,NaN,Basic Statistics,3,3,4
4,A.MTH 1109,NaN,"Properties of Matter, Wave and\rSound",3,3,4


In [5]:
import re

def remove_new_line(document):
    document = document.strip()
    doc = document.replace("\n", " ")
    return doc
    
def remove_special(document):
    doc = re.sub("\- ([a-zA-Z]+)", r"------",document)
    doc = doc.strip()
    doc = doc.replace('  ', '-')
    doc = doc.replace('  - ','-')
    doc = doc.replace(' - ','-')
    doc = doc.replace(' -', '-')
    doc = doc.replace(' –', '-')
    doc = doc.replace('- ', '-')
    doc = doc.replace('--','-')
    return doc

def get_semester(string):
    s = int(string[-4:-2])
    if s == 11:
        return 1
    if s == 12:
        return 2
    if s == 21:
        return 3
    if s == 22:
        return 4
    if s == 31:
        return 5
    if s == 32:
        return 6
    if s == 41:
        return 7
    if s == 44:
        return 8

In [8]:
df['Name'] = df["Name"].apply(remove_new_line)
df['Code'] = df['Code'].apply(remove_special)
df['Semester'] = df['Code'].apply(get_semester)

KeyError: 'Name'

In [6]:
def gen_cypher(obj:dict, node_name= '', node_alias=1):
    st = '{'
    for k, v in obj.items():
        if type(v) is int:
            st += f' {k}: {v},'
        else:
            st += f' {k}: "{v}",'
    st = st[:-1]
    st += ' }'
    s = f'MERGE ({node_alias}:{node_name} {st})'
    return {f"{node_alias}": s}

def gen_relations(child:str, parent:str, relations:str):
    return f"MERGE  ({child})-[:{relations}]->({parent})"

In [31]:
# new_df = df.iloc[:17, :]
# new_df['Credits'] = new_df['Credits'].astype('int')
# new_df['Semester'] = new_df['Semester'].astype('int')

new_df = pd.read_csv('211.csv')
new_df.rename(columns={"Total Marks": "Marks"}, inplace=True)
new_df['Semester'] = new_df['Code'].apply(get_semester)
dics = new_df.T.to_dict('dict')
new_df.head()

,Code,Name,Credits,Marks,Duration,Semester
0,A.MTH-1101,Basic Algebra,3,100,4,1
1,A.MTH-1103,Set Theory,3,100,4,1
2,A.MTH-1105,Calculus-I,3,100,4,1
3,A.MTH-1107,Basic Statistics,3,100,4,1
4,A.MTH-1109,"Properties of Matter, Wave and Sound",3,100,4,1


In [8]:
cypher = []
sem =0
for i in range(1,5):
    for j in range(1,3):
        sem += 1
        year = i
        term = j
        cypher.append(gen_cypher({'Semister': sem, "Year": year, "Term":term}, node_name="Semister", node_alias=f"S{sem}"))
# ' '.join(cypher)

In [32]:
cypher = {}
query= []
sem =0
for i in range(1,5):
    for j in range(1,3):
        sem += 1
        year = i
        term = j
        query.append(gen_cypher({'Semister': sem, "Name": f"Semester-{sem}", "Year": year, "Term":term}, node_name="Semester", node_alias=f"S{sem}")[f"S{sem}"])

# ' '.join(cypher)

for key, val in dics.items():
    query.append(gen_cypher(val, node_name='Course', node_alias=f"K{key}")[f"K{key}"])
    query.append(gen_relations(f"K{key}", f"S{val['Semester']}", 'COURSE_OF'))
    query.append(gen_relations(f"S{val['Semester']}", f"K{key}", 'HAS_COURSE'))
    cypher[f"{val['Name'].lower()}"] = f"K{key}"
# ' '. join(cypher)
' '. join(query)


'MERGE (S1:Semester { Semister: 1, Name: "Semester-1", Year: 1, Term: 1 }) MERGE (S2:Semester { Semister: 2, Name: "Semester-2", Year: 1, Term: 2 }) MERGE (S3:Semester { Semister: 3, Name: "Semester-3", Year: 2, Term: 1 }) MERGE (S4:Semester { Semister: 4, Name: "Semester-4", Year: 2, Term: 2 }) MERGE (S5:Semester { Semister: 5, Name: "Semester-5", Year: 3, Term: 1 }) MERGE (S6:Semester { Semister: 6, Name: "Semester-6", Year: 3, Term: 2 }) MERGE (S7:Semester { Semister: 7, Name: "Semester-7", Year: 4, Term: 1 }) MERGE (S8:Semester { Semister: 8, Name: "Semester-8", Year: 4, Term: 2 }) MERGE (K0:Course { Code: "A.MTH-1101", Name: "Basic Algebra", Credits: 3, Marks: 100, Duration: 4, Semester: 1 }) MERGE  (K0)-[:COURSE_OF]->(S1) MERGE  (S1)-[:HAS_COURSE]->(K0) MERGE (K1:Course { Code: "A.MTH-1103", Name: "Set Theory", Credits: 3, Marks: 100, Duration: 4, Semester: 1 }) MERGE  (K1)-[:COURSE_OF]->(S1) MERGE  (S1)-[:HAS_COURSE]->(K1) MERGE (K2:Course { Code: "A.MTH-1105", Name: "Calculus-I

In [16]:
df1.to_csv('only_course_code.csv')

In [34]:
# from pdfminer.high_level import extract_text

# text = extract_text(file_name)
# with open('first.txt', 'w') as file:
#     file.write(text)


In [31]:
with open('first.txt', 'r') as file:
    data = file.readlines()

In [28]:
# for line in data:
#     if 'A.MTH  1101' in line and 'Basic Algebra' in line:
#         print(line)
cypher

{'basic algebra': 'K0',
 'set theory': 'K1',
 'calculus-i': 'K2',
 'basic statistics': 'K3',
 'properties of matter, wave and sound': 'K4',
 'basic computer': 'K5',
 'basic english skills and commercial correspondence': 'K6',
 'calculus-ii': 'K7',
 'linear algebra': 'K8',
 'vector analysis': 'K9',
 'probability theory & distribution function': 'K10',
 'electricity and magnetism': 'K11',
 'calculus-iii': 'K12',
 'co-ordinate geometry of two dimensions': 'K13',
 'co-ordinate geometry of three dimensions': 'K14',
 'ordinary differential equations': 'K15',
 'statistical methods & demography': 'K16'}

In [33]:
list_topic = {
    "Basic Algebra": {
        "Inequalities": [
            "Order properties of Real Numbers",
            "Average",
            "Mean",
            "Arithmetic Mean",
            "Geometric Mean",
            "Weierstras's Inequality",
            "Cauchy-Schwarz Inequality",
            "Tchebychef's Inequality",
            "Holder's inequality"
        ],
        "Theory of Equations": [
            "Equations and Identities",
            "Basic Ideas of Solutions of Equations with Graphical Representations",
            "Relations between Roots and Coefficient of the nth Degree Equations",
            "Synthetic division",
            "Descartes’ Rule of Signs",
            "Newton's Method",
            "Reciprocal Equations and Sturm's Theorem",
            "Position of Roots"
        ],
        "Series": [
            "Basic definitions of series",
            "Techniques of summing up series",
            "Test for Convergence and Divergence of Series (Less emphasis ion proofs of Theorems)"
        ],
        "Determinants and Matrices": [
            "Elementary properties of Determinants and Matrices",
            "Techniques of Solving linear non-homogeneous and homogeneous Equations",
            "Cramer's Rule",
            "Ranks of Matrices",
            "Echelon Matrices"
        ]
    },
    "Set Theory": {
        "Basic concept of set": [
            "Set operations",
            "Power set",
            "Van diagram",
            "Universal Set etc."
        ],
        "Relations and functions": [
            "Equivalence Relations",
            "Distinction between Relations and Functions",
            "Fundamental Theorem of Equivalence Relations",
            "Partition",
            "Functions: Surjective, Injective, bijective functions",
            "Inverse function",
            "One-one function",
            "Onto function",
            "Characteristic function",
            "Set function",
            "Restriction function",
            "Extension function",
            "Real valued function"
        ],
        "Natural numbers and the axiom of infinity": [],
        "Cardinality": [
            "Countability Cardinal Numbers",
            "Cantor's Theorem",
            "Partially Ordered Sets",
            "Well-Ordered Sets",
            "Ordinal Numbers"
        ],
        "Logic and Language": [
            "Arguments containing compound statements",
            "Method of Deduction",
            "Quantification theory",
            "The logic of relation",
            "Prepositional calculus"
        ]
    },
    "Calculus-i": {
        "Functions": [
            "Basic ideas of the Real and Complex Number Systems.",
            "Functions of a Single Variable and its domain and ranges.",
            "Graphical representation of standard functions (e.g., polynomials, trigonometric and exponential "
            "functions)."
        ],
        "Limit and Continuity": [
            "Existence of limits.",
            "Definition and elementary properties of continuous functions of a single variable.",
            "Statement and verifications of standard theorems by suitable examples (no proofs)."
        ],
        "Differentiation": [
            "Techniques of finding derivatives of functions of a single variable.",
            "Geometrical interpretation of derivatives.",
            "Fundamental theorems (Rolle's theorem, Mean Value theorem, Taylor's theorem, etc.).",
            "Successive differentiations, Leibniz's theorem.",
            "L'Hopital's rule for indeterminate forms (no proof of theorem).",
            "Basic ideas of functions of two and three variables: partial and total derivatives."
        ],
        "Integration (Indefinite and Definite Integrals)": [
            "Integrations of functions of a single variable.",
            "Integrals as antiderivatives.",
            "Techniques of integration of standard forms.",
            "Integrals as limits of sums.",
            "Geometrical meaning of definite integrals."
        ],
        "Definite Integrals": [
            "Elementary properties of definite integrals.",
            "Basic ideas of double and triple integration.",
            "Gamma and beta function.",
            "Fundamental theorems of calculus reduction formula."
        ]
    },

    "Basic Statistics": {
        "Introduction to Statistics": [
            "Definition, background, and scope of Statistics",
            "Meaning, characteristic features, and uses of Statistics",
            "Presentation of statistical data (Graphs, Charts, and Diagrams)",
            "Qualitative and Quantitative data"
        ],
        "Measures of Central Tendency and Dispersion": [
            "Measure of location (Important averages and their comparative study)",
            "Central tendency",
            "Measure of dispersion",
            "Moments of frequency distribution",
            "Skewness and Kurtosis and their measurements"
        ],
        "Bivariate Universe and Correlation": [
            "Coefficient of simple correlation",
            "Bivariate frequency distribution",
            "Simple Regression line and correlation Coefficient",
            "Rank correlation",
            "Comparison of regression analysis with correlation analysis",
            "Useful theorem in regression and correlation analysis",
            "Correlation ratio"
        ],
        "Multiple Regressions and Partial Correlation": [
            "Multiple regressions",
            "Partial correlation",
            "Multiple correlations",
            "Estimation of multiple regression coefficient",
            "Coefficient determination"
        ]
    },
    "Properties of Matter, Wave and Sound": {
        'Gravitation and Gravity': [
            'Law of universal gravitation',
            'Inertial and gravitational masses',
            'Gravitational intensity and potential',
            'Gravitational potential energy',
            'Gravitational effects of symmetrical distribution of matter',
            'Motion of planets and satellites',
            'Variation of acceleration due to gravity'
        ],
        'Elasticity': [
            'Introduction',
            'stress',
            'strain',
            "Hook's law",
            'Elastic modulli',
            'relations between elastic constants and their determination',
            'Bending of beams: Cantilever'
        ],
        'Surface Tension': [
            'Surface energy',
            'surface tension and molecular forces',
            'pressure within a curved film',
            'capillarity',
            'surface tension and temperature',
            'measurements of surface tension'
        ],
        'Wave in elastic media': [
            'Plane compressional waves in a fluid',
            'superposition principle',
            'wave velocity',
            'Power and Intensity in wave motion',
            'Stationary waves',
            "Doppler effects and its applications in Sound and Light"
        ],
        'Vibrations': [
            'Strings, bars and air-columns',
            'Laws of Transverse vibrations',
            "Sonometer, Melde's experiments",
            'resonance',
            'organ pipes',
            'Measurement of Velocity of Sound: Resonance air column method',
            "Kundt's tube method",
            'Acoustical Instruments'
        ]
    },

    "Calculus-ii": {
        "Applications of Differentiation": [
            "Extrema (Maxima or Minima) of Subsets of the Real Number System",
            "Real Valued Functions of Single Variable and Its Extrema",
            "Narrative extrema Problems and Related Rates Based on Elementary Theory"
        ],
        "Further Applications of Differentiation": [
            "Tangent Line Approximation, Differentials, Tangents, Normals, Curvature and Asymptotes",
            "Point of Inflection, Increasing and Decreasing, Convexity and Concavity of functions",
            "Curve Tracing, Polynomials and Rational Functions"
        ],
        "Applications of Integration": [
            "Reduction formula",
            "Lengths of Curves",
            "Areas of Plane Regions",
            "Areas of Surface of Solids of Revolution",
            "Volumes of Solids of Revolution",
            "Moments and Centroids of Plane regions and solids of Revolution"
        ]
    },
    "Linear Algebra": {
        "Concept of Vector Spaces": [
            "Basic concepts of Groups and Fields (Real Numbers, Complex Numbers, and Rational Numbers)",
            "Definition of vector spaces, subspaces, and basic theorems"
        ],
        "Basis and Dimension": [
            "Linear combinations of vectors",
            "Spanning sets",
            "Linear dependence and independence of vectors",
            "Basis and dimension of a vector space",
            "Finite dimensional vector spaces",
            "Sums and direct sums of subspaces"
        ],
        "Linear Transformations": [
            "Linear transformation",
            "Matrix of a linear transformation",
            "Kernel and image of a linear transformation",
            "Rank and nullity of a linear transformation",
            "Non-singular linear transformation",
            "Linear functionals",
            "Dual spaces",
            "Dual bases"
        ],
        "Diagonalization of Matrices": [
            "Eigenvalues and eigenvectors",
            "Diagonalization of matrices",
            "Minimal polynomial of a matrix",
            "Cayley-Hamilton theorem",
            "Diagonalization of symmetric matrices"
        ],
        "Inner Product Spaces, Norms": [
            "Inner products",
            "Norms",
            "Orthogonality",
            "Gram-Schmidt orthogonalization process",
            "Bilinear forms",
            "Symmetric, antisymmetric, and quadratic forms",
            "Related theorems"
        ]
    },
    "Vector Analysis": {
        "Vector Algebra and Vector Calculus": [
            "Vector Algebra",
            "Scalar and Vector Fields",
            "Differentiation of Vector Functions"
        ],
        "Gradient Divergence and curl": [
            "Definitions",
            "Geometrical meanings",
            "Applications"
        ],
        "Vector integration": [
            "Definition",
            "Integration along a line, closed curve, surface and volume"
        ],
        "Vector integral theorems": [
            "Green’s, Gauss and Stokes' Theorems",
            "Applications and verifications"
        ],
        "Curvilinear Co-ordinates": [
            "General Curvilinear Co-ordinates",
            "Orthogonal Curvilinear Co-ordinates",
            "Spherical Polar Co-ordinates",
            "Cylindrical Co-ordinates"
        ],
        "Differential Operators in General Curvilinear Co-ordinates and Orthogonal Curvilinear Co-ordinates": [
            "Line, Surface and Volume Elements in Curvilinear Co-ordinates"
        ]
    },
    "Probability Theory and Distribution Function": {
        "Probability Theory": {
            "Definition and meaning of probability": [
                "Mathematical and statistical definition of probability.",
                "Sample space, simple and compound events."
            ],
            "Additive and multiplicative law of probability": [
                "Marginal and conditional probability.",
                "Bayes' Theorem and its applications."
            ],
            "Random variable and its probability distribution": [
                "Probability function, probability density function, distribution function.",
                "Marginal and conditional distribution function.",
                "Joint probability density function.",
                "Marginal probability density function.",
                "Conditional probability density function."
            ],
            "Some Theoretical Distributions": [
                "Mathematical expectation.",
                "Theorem of expectation.",
                "Covariance.",
                "Variance of a linear combination of variates.",
                "Relation between moments and cumulants.",
                "Basic probability distributions (Binomial, Poisson, and Normal)."
            ],
            "Sampling Distributions": [
                "Definition, uses, and properties.",
                "F, t, and normal distributions.",
                "Test based on F, t, and normal distributions.",
                "Contingency table and its applications."
            ]
        }
    },
    "Electricity and Magnetism": {
        "Electrostatics": [
            "Electric dipole",
            "Electric field due to a dipole",
            "Dipole on external electric field",
            "Gauss's law and its applications"
        ],
        "Capacitors": [
            "Parallel plate capacitors with dielectrics",
            "Dielectrics and Gauss's law",
            "Susceptibility, permittivity, and dielectric constant",
            "Energy stored in an electric field"
        ],
        "Electric Current": [
            "Electron theory of conductivity",
            "Conductors, semiconductors, and insulators",
            "Superconductors",
            "Current and current density",
            "Kirchhoff's law and its applications"
        ],
        "Magnetism": [
            "Magnetic dipole",
            "Mutual potential energy of two small magnets",
            "Magnetic shell",
            "Energy in a magnetic field",
            "Magnetometers"
        ],
        "Electromagnetic Induction": [
            "Faraday's experiment",
            "Faraday's law",
            "Ampere's law",
            "Motional EMF",
            "Self and mutual inductance",
            "Galvanometers - moving cell ballistic and deadbeat types"
        ],
        "Thermoelectricity": [
            "Thermal EMF",
            "Seebeck, Peltier, and Thomson Effects",
            "Laws of thermal EMFs",
            "Thermoelectric power"
        ],
        "DC and AC Circuits": [
            "DC circuits with 1-R, RC, LC, and LCR in series",
            "AC circuits with LR, RC, LC, and LCR in series"
        ]
    },
    "Calculus-iii": {
        "Functions of Several Variables": ["Functions of Two and Three Variables",
                                           "Real Valued and Vector Valued Functions",
                                           "Scalar Fields and Vector Fields"],
        "Limit and Continuity of Functions of Several Variables": ["Partial Differentiation", "Total Differentiation",
                                                                   "Geometrical Meaning of Partial Derivative"],
        "Extrema of Functions of Several Variables": ["First Derivative Test", "Second Derivative Test",
                                                      "Lagrange's Multiplier Method", "Taylor's Theorem"],
        "Integration of Functions of Several Variables": ["Line Integrals", "Double and Triple Integration",
                                                          "Green's Theorem", "Gauss's Divergence Theorem",
                                                          "Stokes' Theorem"],
        "Improper Integrals": ["Definition and Classification of Improper Integrals",
                               "Tests for Convergence and Divergence of Improper Integrals", "Gamma and Beta Functions"]
    },
    "co-ordinate geometry of two dimensions": {
        "Coordinate Systems": ["Cartesian and Polar Coordinates", "Parameters", "Changes of Axes", "Standard Equations",
                               "Parametric Representations"],
        "General Equations of Second Degree": ["Standard Forms",
                                               "Reduction of General Equation of Second Degree into Standard Forms"],
        "Pair of Straight Lines": ["General Equation of Second Degree representing a Pair of Straight Lines",
                                   "General Properties of Pair of Straight lines"],
        "Circles and System of Circles": [],
        "Properties of Conic Sections": ["Parabola", "Ellipse", "Hyperbola", "Rectangular Hyperbola"]
    },
    "co-ordinate geometry of three dimensions": {
        "Coordinate Systems in Three Dimensions": [
            "Display of Solid Figures",
            "Rectangular Coordinates: Direction Cosines & Direction Ratios",
            "Projections",
            "Angle between two Lines"
        ],
        "Planes and Straight lines": [
            "Planes: Constants in the Equations of Plane, Different Forms of the Equation of a Plane and Conversions, "
            "Angle between two planes",
            "Lines: Different Forms of Equations of Lines and Conversions, Skew Lines, Constants in the Equation of a "
            "Line, Angle between a Line and a Plane",
            "A Line in a Plane Through a Line",
            "Coplanar Lines",
            "Shortest Distance"
        ],
        "Spheres": [
            "Equations of Spheres",
            "A Plane and Sphere",
            "A Line and Sphere",
            "Circle in three dimensions",
            "Plane of Contact",
            "Tangent Planes",
            "Polar Planes",
            "Angle of Intersection of Two Spheres",
            "Condition of Orthogonality",
            "Radical Line",
            "Plane and Centres",
            "Coaxial Spheres"
        ],
        "Cones and Cylinders": [
            "Generators",
            "Necessary Condition for a General Eq. of Second Degree to represent a cone",
            "Right Circular Cones",
            "Equations of Cylinders",
            "Circular Cylinders",
            "Enveloping Cylinders and Enveloping Cones"
        ],
        "The General Equation of Second Degree": [
            "Conicoids",
            "Reduction to Standard Forms",
            "Conicoids of Revolution",
            "Ellipsoids, Paraboloids & Hyperboloids"
        ]
    },
    "Ordinary Differential Equations": {
        "Introductory Concepts": [
            "Definitions of Ordinary Differential Equations (ODE)",
            "Construction of ODEs",
            "Geometrical Meanings of ODEs and their Solutions",
            "Classification of ODEs",
            "Initial Value Problems"
        ],
        "Techniques of Solving ODE's": [
            "First Order and 1st Degree Differential Equations",
            "Trajectories",
            "Linear Differential Equation with Constant Coefficients",
            "Homogeneous Linear Differential Equations and their Applications",
            "Equation of 1st Order but not of 1st Degree",
            "Singular Solutions"
        ],
        "Further Techniques": [
            "Method of Variation of Parameters",
            "Method of Undetermined Coefficients",
            "Reduction of Linear Differential Equations to a First Order System"
        ],
        "Higher Order Linear Differential Equations": [
            "Initial Value Problem (IVP)",
            "Boundary Value Problem (BVP)",
            "The Wronskians and its Properties",
            "Reduction of Order",
            "A Formula for the Wronskian",
            "Exact Differential Equations"
        ]
    },
    "Statistical Methods & Demography": {
        "Economical Statistics": [
            "Index number",
            "simple price index",
            "simple aggregative-index",
            "weight indices",
            "cost of living indices",
        ],
        "Time series": [
            "concept of time series analysis",
            "components of time-series and their relationship",
            "measurement of trend by different methods like method of free-hand curve, method of moving average and "
            "method of least square"
            ,
            "determination of seasonal fluctuation",
            "simple averages ratio to moving average methods",
            "ratio to link relative"
        ],
        "Vital Statistics": [
            "sex, marital status, economic activity, etc., and their usefulness in demographic analysis",
            "Sources of demographic data population census, vital registration, surveys and population composition by "
            "age"
            ,
            "Rates and ratio",
            "Examples of above statistics in relation to Bangladesh population"
        ],
        "Fertility": [
            "Crude and age-sex specific birth rates.",
            "Central and total fertility rates.",
            "Standardised rates and their relative merits and demerits.",
            "GRR and NRR.",
        ],
        "Mortality": [
            "Crude and age-sex specific death rates.",
            "Standardised death rate.",
            "Infant mortality rate.",
            "Census of deaths.",
            "Life table (complete and abridged) and their uses.",
        ],
        "Educational Statistics & Quality Control": [
            "Concept of statistical quality control.",
            "The importance of statistical quality control.",
            "Schwart control chart technique.",
            "Control chart (X proportion and range) and uses.",
            "Merits and demerits of control chart.",
            "Soeing (T score and Z score) validity, reliability, I.Q., etc.",
        ],
    },
    "Tensor Analysis": {
        "Linear Transformation": ["General Linear Transformations",
                                  "Orthogonal Transformation and Admissible Transformations"],
        "Tensor Algebra": ["Riemannian space", "Invariant, Contravariant and Covariant Vectors",
                           "Tensors of Different Ranks and Types",
                           "Symmetric and Skew Symmetric Tensors in Euclidean Space and in Riemannian Space",
                           "Addition and Subtraction of Tensors", "Outer Product and Inner Product of Tensors",
                           "Contraction and Quotient law", "Metric Tensor", "Index Raising and Lowering."],
        "Tensor Calculus and Riemannian metric": ["Christoffel Symbols of First & Second kind", "Parallel Displacement",
                                                  "Covariant Differentiation of Tensors",
                                                  "Metric Affinity and the Christoffel symbols in terms of the Metric "
                                                  "Tensor",
                                                  "Basic Concepts of Riemann-Christoffel Curvature Tensor",
                                                  "Ricci Tensor", "Bianchi identity",
                                                  "Geodesics and Equations of Geodesics."],
        "Applications of Tensor analysis to electromagnetic theory and relativity.": []
    }
}


In [47]:


nwe = []
t = 0
s = 0
for key, value in list_topic.items():
    print(key)

    for main_topic, main_topic_value in value.items():
        parent = cypher.get(key.lower(), None)
        if parent:
            t += 1

            topic = {"Name": main_topic.lower() }
            nwe.append(gen_cypher(topic, node_name="Topic", node_alias=f"T{t}")[f"T{t}"])
            nwe.append(gen_relations(f"T{t}", parent, 'TOPIC_OF'))
            nwe.append(gen_relations(parent, f"T{t}", 'HAS_TOPIC'))
            for lower_topic in main_topic_value:
                ltopic = {"Name": lower_topic.lower() }
                nwe.append(gen_cypher(ltopic, node_name="SubTopic", node_alias=f"ST{t}{s}")[f"ST{t}{s}"])
                nwe.append(gen_relations(f"ST{t}{s}", f"T{t}", 'SUBTOPIC_OF'))
                nwe.append(gen_relations(f"T{t}", f"ST{t}{s}", 'HAS_SUBTOPIC'))
                s += 1


Basic Algebra
Set Theory
Calculus-i
Basic Statistics
Properties of Matter, Wave and Sound
Calculus-ii
Linear Algebra
Vector Analysis
Probability Theory and Distribution Function
Electricity and Magnetism
Calculus-iii
co-ordinate geometry of two dimensions
co-ordinate geometry of three dimensions
Ordinary Differential Equations
Statistical Methods & Demography
Tensor Analysis


In [48]:
with open('./ontology.cypher', 'w') as file:
    file.write('\n'.join(query + nwe))